# 🚀 대규모 민원 이미지 AI 학습 (10,000장 버전)
이 노트북은 5가지 카테고리의 민원 이미지를 학습시켜 YOLOv8 모델을 생성합니다.

### 📌 작업 순서
1. 로컬에서 생성한 `large_dataset.zip`을 구글 드라이브의 `AI_PROJECT` 폴더에 업로드합니다.
2. 아래 셀들을 순서대로 실행합니다.

## 1. 구글 드라이브 연결 및 데이터 준비

In [ ]:
from google.colab import drive
import os

# 1. 구글 드라이브 마운트
drive.mount('/content/drive')

# 2. 데이터셋 복사 및 압축 해제 (경로를 본인의 업로드 경로에 맞게 확인하세요)
# 만약 AI_PROJECT 폴더가 없다면 직접 수정이 필요합니다.
dataset_zip = '/content/drive/MyDrive/AI_PROJECT/large_dataset.zip'

if os.path.exists(dataset_zip):
    !cp "{dataset_zip}" /content/
    !unzip -q /content/large_dataset.zip -d /content/dataset
    print("✅ 데이터셋 준비 완료!")
else:
    print("❌ 구글 드라이브에서 'large_dataset.zip'을 찾을 수 없습니다.")
    print("경로를 다시 확인해 주세요: ", dataset_zip)

## 2. 학습 환경 설정

In [ ]:
# Ultralytics 패키지 설치
!pip install ultralytics

import shutil
from ultralytics import YOLO
from IPython.display import display, Image

# GPU 사용 가능 여부 확인
import torch
print(f"사용 가능한 GPU: {torch.cuda.get_device_name(0)}") if torch.cuda.is_available() else print("⚠️ GPU가 활성화되지 않았습니다. 런타임 유형을 수정하세요.")

## 3. 데이터 검증 (옵션)
데이터가 제대로 압축 해제되었는지 확인합니다.

In [ ]:
import glob
train_images = glob.glob('/content/dataset/images/train/*')
val_images = glob.glob('/content/dataset/images/val/*')
print(f"학습용 이미지 수: {len(train_images)}")
print(f"검증용 이미지 수: {len(val_images)}")

## 4. AI 모델 학습 시작
10,000장의 데이터를 기반으로 본격적인 학습을 진행합니다.

In [ ]:
# 기본 모델 로드 (빠르고 가벼운 YOLOv8n 사용)
model = YOLO('yolov8n.pt')

# 학습 실행
results = model.train(
    data='/content/dataset/data.yaml', 
    epochs=50,          # 데이터가 많을 때는 30~50회면 충분합니다.
    imgsz=640,          # 이미지 크기
    device=0,           # GPU 사용
    batch=16,           # GPU 메모리에 따라 조정 (T4 기준 16 권장)
    patience=10,        # 성능 향상이 없으면 10에폭 후 자동 종료
    name='civic_analysis' # 결과 저장 폴더 이름
)

## 5. 학습 결과 확인 및 모델 다운로드

In [ ]:
from google.colab import files
import glob

# 가장 마지막에 생성된 학습 결과의 best.pt 찾기
best_weights = glob.glob('/content/runs/detect/civic_analysis*/weights/best.pt')

if best_weights:
    print(f"✅ 최적의 모델 파일을 찾았습니다: {best_weights[-1]}")
    files.download(best_weights[-1])
else:
    print("❌ 모델 파일을 찾을 수 없습니다. 학습이 정상적으로 완료되었는지 확인하세요.")